In [1]:
! pip install pytorch_tabnet gdown -q


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, roc_auc_score, average_precision_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
import time
import gdown


In [4]:

!gdown https://drive.google.com/uc?id=1m2zrxuamOjejPA_283824cvmtQ6EzRSM
df = pd.read_csv('/kaggle/working/Dataset_ previous values_gpbyID+NoShift+F.Engv3.0.csv')
df


Downloading...
From (original): https://drive.google.com/uc?id=1m2zrxuamOjejPA_283824cvmtQ6EzRSM
From (redirected): https://drive.google.com/uc?id=1m2zrxuamOjejPA_283824cvmtQ6EzRSM&confirm=t&uuid=6de659f1-93f7-4405-9534-00a7d102d02d
To: /kaggle/working/Dataset_ previous values_gpbyID+NoShift+F.Engv3.0.csv
100%|█████████████████████████████████████████| 420M/420M [00:01<00:00, 259MB/s]


/tmp/ipykernel_13/3837367766.py:2: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/working/Dataset_ previous values_gpbyID+NoShift+F.Engv3.0.csv')


,Unnamed: 0,...1,Hour,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,...,Patient_ID,Incomplete_qSOFA_Score,SOFA_Score,SIRS,NEWS_Score,Apache_II_Score,BUN_Creatinine_Ratio,Shock_Index,Bilirubin_Creatinine_Ratio,datetime
0,0,0,0,65.0,100.0,35.78,129.0,72.0,69.0,16.5,...,17072,0,2,0,0,16,28.750000,0.503876,0.875000,2019-01-01 01:00:00
1,1,1,1,65.0,100.0,35.78,129.0,72.0,69.0,16.5,...,17072,0,2,0,0,16,28.750000,0.503876,0.875000,2019-01-01 02:00:00
2,2,2,2,78.0,100.0,35.78,129.0,42.5,69.0,16.5,...,17072,0,5,0,0,18,28.750000,0.604651,0.875000,2019-01-01 03:00:00
3,3,3,3,73.0,100.0,35.78,129.0,42.5,69.0,17.0,...,17072,0,5,0,0,18,28.750000,0.565891,0.875000,2019-01-01 04:00:00
4,4,4,4,70.0,100.0,35.78,129.0,74.0,69.0,14.0,...,17072,0,2,0,0,14,28.750000,0.542636,0.875000,2019-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552205,1552205,21,21,83.0,99.0,36.10,121.0,77.0,54.0,22.0,...,113911,1,2,0,1,10,45.945946,0.685950,1.891892,NaN
1552206,1552206,22,22,80.0,92.0,36.10,102.0,73.0,51.0,24.0,...,113911,1,2,0,1,10,45.945946,0.784314,1.891892,NaN
1552207,1552207,23,23,95.0,97.0,36.70,128.5,83.0,58.5,25.0,...,113911,1,2,1,4,11,45.945946,0.739300,1.891892,NaN
1552208,1552208,24,24,104.0,99.0,36.70,127.0,85.0,59.0,24.0,...,113911,1,2,1,2,10,45.945946,0.818898,1.891892,NaN


In [5]:


# Dropping unnecessary columns
df.drop(['Unnamed: 0','...1','Hour', 'Unit1', 'Unit2','Apache_II_Score','Patient_ID', 'SOFA_Score', 'SIRS', 'Bilirubin_Creatinine_Ratio'], axis=1, inplace=True)


In [6]:


# Data preparation
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['weekday'] = df['datetime'].dt.weekday
df.drop('datetime', axis=1, inplace=True)

# Fill NaN values
df.fillna(-1, inplace=True)


In [7]:

# Addressing class imbalance
class_labels = np.unique(df['SepsisLabel'])
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=df['SepsisLabel'].values)
class_weight_dict = {class_label: weight for class_label, weight in zip(class_labels, class_weights)}
print("Class Weight Dictionary:", class_weight_dict)

# Split the data into features (X) and target (y)
X = df.drop(['SepsisLabel'], axis=1)
y = df['SepsisLabel'].values


Class Weight Dictionary: {0: 0.5091570261380023, 1: 27.801440034388882}


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [9]:

# Create a TabNet classifier
clf = TabNetClassifier(
    n_d=64,
    n_a=64,
    n_steps=5,
    gamma=1.5,
    n_independent=2,
    n_shared=2,
    momentum=0.3,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax',
    verbose=1,
    device_name='auto'
)

/usr/local/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [10]:
# Fit the classifier with class weights
try:
  clf.fit(
      X_train=X_train.values,
      y_train=y_train,
      eval_set=[(X_train.values, y_train), (X_test.values, y_test)],
      eval_name=['train', 'test'],
      eval_metric=['auc','accuracy'],
      weights=class_weight_dict,  
      max_epochs=250,
      patience=20,
      batch_size=2500,
      virtual_batch_size=128,
      num_workers=0,
      drop_last=False
  )
except KeyboardInterrupt:
    print("Training interrupted.")


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


epoch 0  | loss: 0.59104 | train_auc: 0.79551 | train_accuracy: 0.73566 | test_auc: 0.79119 | test_accuracy: 0.73439 |  0:04:15s
epoch 1  | loss: 0.49814 | train_auc: 0.89195 | train_accuracy: 0.79889 | test_auc: 0.87203 | test_accuracy: 0.79842 |  0:08:26s
epoch 2  | loss: 0.30528 | train_auc: 0.95796 | train_accuracy: 0.8763  | test_auc: 0.93389 | test_accuracy: 0.87333 |  0:12:42s
epoch 3  | loss: 0.21583 | train_auc: 0.98848 | train_accuracy: 0.94757 | test_auc: 0.97386 | test_accuracy: 0.9445  |  0:16:56s
epoch 4  | loss: 0.12285 | train_auc: 0.99393 | train_accuracy: 0.95413 | test_auc: 0.98063 | test_accuracy: 0.95083 |  0:21:10s
epoch 5  | loss: 0.09189 | train_auc: 0.99541 | train_accuracy: 0.96431 | test_auc: 0.98263 | test_accuracy: 0.96026 |  0:25:23s
epoch 6  | loss: 0.07634 | train_auc: 0.99641 | train_accuracy: 0.97247 | test_auc: 0.98461 | test_accuracy: 0.96918 |  0:29:36s
epoch 7  | loss: 0.06731 | train_auc: 0.99713 | train_accuracy: 0.98026 | test_auc: 0.98567 | tes

In [11]:


# Prediction on the test set
predictions = clf.predict(X_test.values)
prob_predictions = clf.predict_proba(X_test.values)[:, 1]

# Evaluation metrics
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, prob_predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f2 = fbeta_score(y_test, predictions, beta=2)
classification_report_output = classification_report(y_test, predictions)
confusion_matrix_report = confusion_matrix(y_test, predictions)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"F2 Score: {f2:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print("Classification Report:")
print(classification_report_output)
print("Confusion Matrix:")
print(confusion_matrix_report)

Accuracy: 0.9876
Precision: 0.6000
Recall: 0.9370
F1 Score: 0.7315
F2 Score: 0.8423
ROC AUC: 0.9875
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    304859
           1       0.60      0.94      0.73      5583

    accuracy                           0.99    310442
   macro avg       0.80      0.96      0.86    310442
weighted avg       0.99      0.99      0.99    310442

Confusion Matrix:
[[301371   3488]
 [   352   5231]]


In [13]:
# Feature importance
feature_importances = clf.feature_importances_
feature_importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)
print("Feature Importances:")
print(feature_importances_df)

KeyError: 0